## 进入 MMEditing 主目录

In [2]:
import os
os.chdir('mmediting')
os.listdir()

['.git',
 '.circleci',
 '.dele.yml',
 '.dev_scripts',
 '.github',
 '.gitignore',
 '.owners.yml',
 '.pre-commit-config.yaml',
 '.pylintrc',
 '.readthedocs.yml',
 'CITATION.cff',
 'LICENSE',
 'MANIFEST.in',
 'README.md',
 'README_zh-CN.md',
 'configs',
 'demo',
 'docker',
 'docs',
 'mmedit',
 'model-index.yml',
 'projects',
 'requirements.txt',
 'requirements',
 'setup.cfg',
 'setup.py',
 'tests',
 'tools',
 'mmedit.egg-info',
 'outputs',
 'data',
 'checkpoints',
 'work_dirs',
 'indexnet_mobv2_1x16_78k_comp1k_SAD-45.6_20200618_173817-26dd258d.pth',
 'indexnet_mobv2_1x16_78k_comp1k_SAD-45.6_20200618_173817-26dd258d.pth.1',
 'indexnet_mobv2_1x16_78k_comp1k_SAD-45.6_20200618_173817-26dd258d.pth.2']

## 检查推理支持的任务和模型

在编辑中有多种任务类型:抠图、修补、视频超分辨率、图像超分辨率、图像转图像、无条件GANs、有条件GANs、视频插帧。

我们为每个任务提供了一些模型。所有可用的模型和任务都可以打印出来。

In [20]:
from mmedit.edit import MMEdit

# print all supported models for inference.
inference_supported_models = MMEdit.get_inference_supported_models()
print('all supported models:')
print(inference_supported_models)

# print all supported tasks for inference.
supported_tasks = MMEdit.get_inference_supported_tasks()
print('all supported tasks:')
print(supported_tasks)

# print all supported models for one task, take image translation for example.
task_supported_models = MMEdit.get_task_supported_models('Image2Image')
print('translation models:')
print(task_supported_models)

all supported models:
['biggan', 'styleganv1', 'gca', 'global_local', 'aot_gan', 'pix2pix', 'esrgan', 'flavr', 'cain', 'edvr', 'disco_diffusion', 'eg3d']
all supported tasks:
['Image2Image', 'Text2Image, Image2Image, diffusion', '3D-aware Generation', 'Matting', 'Conditional GANs', 'Video Interpolation', 'Video Super-Resolution', 'Inpainting', 'Unconditional GANs', 'Image Super-Resolution']
translation models:
['pix2pix']


## 下载图片和视频

In [8]:
# 创建存放素材的文件夹
!mkdir -p ./resources/input/translation
!mkdir -p ./resources/output/translation

In [19]:
# 查看所有资源
!python demo/download_inference_resources.py --print-all
# 查看所有任务类型
!python demo/download_inference_resources.py --print-task-type
# 查看特定任务的资源
!python demo/download_inference_resources.py --print-task 'Inpainting'
# 将所有资源下载到默认目录'../resources'
!python demo/download_inference_resources.py
# 下载一个任务的资源
!python demo/download_inference_resources.py --task 'Inpainting'
# 下载到您想要的目录
!python demo/download_inference_resources.py --root-dir 'resources'

all inference resources:
Matting
https://download.openmmlab.com/mmediting/resources/input/matting/GT05.jpg
https://download.openmmlab.com/mmediting/resources/input/matting/GT05_trimap.jpg
https://download.openmmlab.com/mmediting/resources/input/matting/readme.md
Inpainting
https://download.openmmlab.com/mmediting/resources/input/inpainting/bbox_mask.png
https://download.openmmlab.com/mmediting/resources/input/inpainting/celeba_test.png
https://download.openmmlab.com/mmediting/resources/input/inpainting/readme.md
Image Super-Resolution
https://download.openmmlab.com/mmediting/resources/input/restoration/000001.png
https://download.openmmlab.com/mmediting/resources/input/restoration/0901x2.png
https://download.openmmlab.com/mmediting/resources/input/restoration/readme.md
Image2Image Translation
https://download.openmmlab.com/mmediting/resources/input/translation/gt_mask_0.png
https://download.openmmlab.com/mmediting/resources/input/translation/readme.md
Video Interpolation
https://downlo

用两行python代码进行推理。 
有两个步骤: 首先，通过预先训练的模型名称创建一个MMEdit实例。
第二，用这个MMEdit实例推断自己的图片。转换后的图像将保存到result_out_dir。

In [18]:
from mmedit.edit import MMEdit

# 创建一个MMEdit实例
editor = MMEdit('pix2pix')
# 推断图像。需要输入图像路径和输出图像路径。
results = editor.infer(img='resources/input/translation/gt_mask_0.png', result_out_dir='resources/output/translation/tutorial_translation_pix2pix_res.jpg')

ValueError: Unknown inferencer task: Image2Image

In [15]:
# plot the result image
import mmcv
import matplotlib.pyplot as plt 
img = mmcv.imread('resources/output/translation/tutorial_translation_pix2pix_res.jpg')
plt.imshow(mmcv.bgr2rgb(img))
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'resources/output/translation/tutorial_translation_pix2pix_res.jpg'